In [ ]:
# contains code from:
# Composer - Apache-2.0 license - Copyright 2022 MosaicML Composer authors

In [1]:
import warnings
import pickle
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW

import transformers
import datasets
from multiprocessing import cpu_count

from torchmetrics.classification import MulticlassAccuracy
from composer.algorithms import FusedLayerNorm, LowPrecisionLayerNorm
from composer.callbacks import SpeedMonitor
from composer.loggers import ProgressBarLogger, InMemoryLogger
from composer.models.huggingface import HuggingFaceModel
from composer.metrics import CrossEntropy
from composer.optim.scheduler import CosineAnnealingWithWarmupScheduler
from composer.utils import module_surgery
from composer import Trainer

from fastxtend.utils import less_random

APEX = BNB = False

from transformers.utils import logging as hf_logging

warnings.simplefilter('ignore')
hf_logging.set_verbosity_error()

In [3]:
model_name = 'mosaicml/mpt-7b'
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

In [4]:
tokenizer

GPTNeoXTokenizerFast(name_or_path='mosaicml/mpt-7b', vocab_size=50254, model_max_length=2048, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True)

In [ ]:
def tokenize_function(sample):
    return tokenizer(
        text=sample['text'],
        padding='max_length',
    )
gutenberg_dataset = datasets.load_dataset('sedthh/gutenberg_english')
tokenized_dataset = gutenberg_dataset.map(tokenize_function,
                                          batched=True,
                                          num_proc=cpu_count(),
                                          batch_size=64)